<a href="https://colab.research.google.com/github/xup5/Computational-Neuroscience-Class/blob/main/Convolutional%20Neural%20Network/Load_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to load an pretrained keras model, and how to get intermediate layer activations.

In [ ]:
!wget https://github.com/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Convolutional%20Neural%20Network/ILSVRC2012_test_00026783.JPEG?raw=true -O ILSVRC2012_test_00026783.JPEG

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow import keras
import tensorflow as tf
import numpy as np

# download the pretrained VGG16 model
model = VGG16(weights='imagenet', include_top=True)

In [ ]:
model.summary()

In [ ]:
# Let's test an image!
# load image
img_path = 'ILSVRC2012_test_00026783.JPEG'
# preprocess
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
# predict one-hot label
y = model.predict(x)

## Get intermediate layer activation

In [ ]:
partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)

In [ ]:
response = partial_model.predict(x)

In [ ]:
response.shape

## In-silico electrophysiology
Let's try to explore some basic properties of the artificial neurons. A good starting point is to find the orientation and spatial period tuning curves.

In [ ]:
# @title A util function to generate stimuli (run this cell)

def makeGrating(size, spatialp, ori=0, phase=0, imsize=224, sharpness=3, contrast=1, annular=0, dtype='uint8', shift=0, center=None):
    """
    Make a square grating.
    size: the full-width-half-maximum of gaussian mask
    which can be thought of as an effective radius.
    spatialp: spatial period.
    ori: orientation, 0 is horizental. 90 is vertical.
    phase: 0-360
    imsize: the image size.
    annular: inside diameter of the donut.
    sharpness: pixels of HMFW of gaussian mask.
    contrast: 0-1.
    """
    ori = ori/180*np.pi
    im = np.ones((imsize,imsize))
    # the last term is to make center phase 0.
    phi = (phase/np.pi*180-2*np.pi/spatialp*imsize/2)
    for x in range(imsize):
        for y in range(imsize):
            im[x,y] = np.sin(2*np.pi/spatialp*((x*np.cos(ori)+y*np.sin(ori))+phi))
    im = im*contrast
    im = (im+1) / 2 * 255
    im = np.repeat(im[:,:,np.newaxis],3,axis=2)
    im = im.astype(dtype)
    return im

In [ ]:
# let see some examples of stimuli
import matplotlib.pyplot as plt

sti1 = makeGrating(size=224, spatialp=20, ori=0)
plt.figure()
plt.imshow(sti1)

sti1 = makeGrating(size=224, spatialp=50, ori=45)
plt.figure()
plt.imshow(sti1)

In [ ]:
# now let's find optimal orientation and spatial period for an artificial neuron.

partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block3_conv3').output)
neuron = 5

oris = [x for x in range(0,180,15)]
periods = [x for x in range(5,35,5)]
responses = np.zeros((len(oris),len(periods)))

for i,ori in enumerate(oris):
  for j,period in enumerate(periods):
    # construct the stimuli
    sti = makeGrating(size=224, spatialp=period, ori=ori)
    sti = np.expand_dims(sti, axis=0)
    sti = preprocess_input(sti)
    # run the model
    responses[i,j] = partial_model.predict(sti, verbose=0)[0,28,28,neuron]

# plot the tuning heatmap
plt.pcolor(oris, periods, responses.T)
plt.colorbar()
plt.xlabel("orientation")
plt.ylabel("spatial period")

## Feature visualization
Now let's visualize this neuron's feature.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

init_val = np.random.normal(size=(1,224,224,3), scale=0.01).astype(np.float32)
im = tf.Variable(init_val)
for epoch in range(100):
  with tf.GradientTape() as tape:
    loss = -tf.reduce_mean(partial_model(im)[0,:,:,neuron])
  gradients = tape.gradient(loss, [im])
  opt.apply_gradients(zip(gradients, [im]))

plt.imshow(im[0,...])

In [ ]:
# let's try another neuron.

partial_model = keras.Model(inputs=model.input, outputs=model.get_layer('block4_conv2').output)
neuron = 5

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

init_val = np.random.normal(size=(1,224,224,3), scale=0.01).astype(np.float32)
im = tf.Variable(init_val)
for epoch in range(100):
  with tf.GradientTape() as tape:
    loss = -tf.reduce_mean(partial_model(im)[0,:,:,neuron])
  gradients = tape.gradient(loss, [im])
  opt.apply_gradients(zip(gradients, [im]))

plt.imshow(im[0,...])

### Note
Above is only a minimal example of visualization. (An immediate but minor problem is we ignored the preprocess.) It often requires several tricks to obtain clear vivid features. An important trick is to use a natural image prior. This is usually done by putting a regulation loss on the Fourier spectrum (natural images are known to have spectrum slop around 1.0).


This distill article is a good learning resource: https://distill.pub/2017/feature-visualization/

A newer study on more tricks: https://arxiv.org/pdf/2201.12961.pdf